In [1]:
import os
import glob
import cv2
import tkinter as tk
from tkinter import filedialog, messagebox
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import uniform_filter
from scipy.stats import ttest_rel
from scipy.stats import ttest_ind
from scipy.stats import pearsonr
from scipy.stats import linregress

In [ ]:
# Part 2 - Occluded Flow

## Importing images

groupimages = 'pathname'

# Get a list of all files in the folder matching the pattern
filePattern = os.path.join(groupimages, "*.bmp")

theFiles = glob.glob(filePattern)

inflated = []
recovery = []

for fullFileName in theFiles:
    baseName = os.path.basename(fullFileName)
    if "recovery" in baseName:
        recovery.append(fullFileName)
    else:
        inflated.append(fullFileName)

for fullFileName in inflated:
    print(f"Now reading (Inflated): {fullFileName}")
    imageArray = cv2.imread(fullFileName)

for fullFileName in recovery:
    print(f"Now reading (Recovery): {fullFileName}")
    imageArray = cv2.imread(fullFileName)

### LSC calculation function

def compute_lsc(image, window_size):
    image = image.astype(np.float32)
    
    # Local mean and standard deviation
    mean = uniform_filter(image, size=window_size)
    mean_sq = uniform_filter(image**2, size=window_size)
    std = np.sqrt(mean_sq - mean**2)
    
    # Calculate speckle contrast map
    lsc = np.where(mean != 0, std / mean, 0)
    
    return lsc

### Defining Region of Interest (ROI) using Thresholding

# used threshold 25 for data analysis
def extract_roi(gray_img, threshold=25):
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray_img, (7, 7), 0)
    
    # Threshold to create a binary mask (adjust threshold as needed)
    _, mask = cv2.threshold(blurred, threshold, 255, cv2.THRESH_BINARY)
    
    # Optional: clean the mask with morphological operations
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8))
    
    # Use the mask to isolate the finger region
    finger_region = cv2.bitwise_and(gray_img, gray_img, mask=mask)
    
    return finger_region, mask

### Calculating the K_values

# List to store (filename, K value) results
infK = []
window_size = 7

# Loop over breath held images
for fullFileName in inflated:
    print(f"Processing: {fullFileName}")
    
    # Read as grayscale
    img = cv2.imread(fullFileName, cv2.IMREAD_GRAYSCALE)

    # Extract finger region and its mask - automatic thresholding
    finger_roi, mask = extract_roi(img)

    # Compute LSC map on finger region
    lsc_map = compute_lsc(finger_roi, window_size)

    # Compute mean K only where mask > 0 (i.e., finger region)
    mean_K = np.mean(lsc_map[mask > 0])
    
    print(f"LSC K value (fingers only): {mean_K:.4f}")
    infK.append(mean_K)

# List to store (filename, K value) results
recK = []
window_size = 7

# Loop over breath held images
for fullFileName in recovery:
    print(f"Processing: {fullFileName}")
    
    # Read as grayscale
    img = cv2.imread(fullFileName, cv2.IMREAD_GRAYSCALE)

    # Extract finger region and its mask
    finger_roi, mask = extract_roi(img)

    # Compute LSC map on finger region
    lsc_map = compute_lsc(finger_roi, window_size)

    # Compute mean K only where mask > 0 (i.e., finger region)
    mean_K = np.mean(lsc_map[mask > 0])
    
    print(f"LSC K value (fingers only): {mean_K:.4f}")
    recK.append(mean_K)

### Performing pooled analysis with other groups data

# Re-read with no header to simplify structure detection
classdata_path = 'pathname'
df_occluded_raw = pd.read_csv(classdata_path, header=None)

# Parse data by detecting new sections (rows with "Time" in col 0 or col 2, depending on layout)
groups = []
current_group = {"time": [], "k_values": []}

for _, row in df_occluded_raw.iterrows():
    # Detect start of new group
    if "Time" in str(row[0]) or "Time" in str(row[2]):
        if current_group["time"] and current_group["k_values"]:
            groups.append(current_group)
            current_group = {"time": [], "k_values": []}
    # Try to parse values from columns 0 and 1
    elif pd.notna(row[0]) and pd.notna(row[1]):
        try:
            time_val = float(row[0])
            k_val = float(row[1])
            current_group["time"].append(time_val)
            current_group["k_values"].append(k_val)
        except ValueError:
            continue
    # Or try to parse values from columns 2 and 3
    elif pd.notna(row[2]) and pd.notna(row[3]):
        try:
            time_val = float(row[2])
            k_val = float(row[3])
            current_group["time"].append(time_val)
            current_group["k_values"].append(k_val)
        except ValueError:
            continue
                    
# removing any values past 100 seconds
if current_group["time"] and current_group["k_values"]:
    groups.append(current_group)
    
for group in groups:
    group["time"] = group["time"][:9]
    group["k_values"] = group["k_values"][:9]
    
    
# Removing invalid group data (I don't think they did the time intervals correctly)
groups.pop(7)

#### Plotting

# Plotting
plt.figure(figsize=(12, 6))

for i, group in enumerate(groups):
    times = group["time"]
    k_values = group["k_values"]
    plt.plot(times, k_values, marker='o')

# Highlight inflation and recovery periods
plt.axvspan(0, 40, color='cyan', alpha=0.2, label='Inflation')
plt.axvline(40, color='blue', linestyle='--', label='Recovery begins')

# Labels and styling
plt.xlabel("Time (s)")
plt.ylabel("LSC K Value")
plt.title("K Value Over Time for All Participants\nWith Pneumatic Cuff Inflation Period Highlighted")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

occluded = []
non_occluded = []

for g in groups:
    if len(g["k_values"]) >= 9:
        occluded.append(g["k_values"][1:5])
        non_occluded.append(g["k_values"][5:9])

# Stack for paired t-test
import numpy as np

occluded_arr = np.array(occluded)
non_occluded_arr = np.array(non_occluded)

# Perform the paired t-test across all participants
t_stat, p_val = ttest_rel(occluded_arr.mean(axis=1), non_occluded_arr.mean(axis=1))

t_stat, p_val